In [29]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio,pytorch_lightning
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

def _IMPORT_(x):
    try:
        exec(x, globals())
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed
pytorch_lightning 1.2.0


In [30]:
###
### Common ###
###

import sys, os, io, time, random, math
import json, base64, requests

_IMPORT_('import numpy as np')
_IMPORT_('import pandas as pd')
_IMPORT_('from tqdm.notebook import tqdm')

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()


###
### Torch ###
###

_IMPORT_('import torch')
_IMPORT_('import torch.nn as nn')
_IMPORT_('import torch.nn.functional as F')
_IMPORT_('import torch.optim as O')
_IMPORT_('from torchvision import models as M')
_IMPORT_('from torchvision import transforms as T')
_IMPORT_('from torch.utils.data import Dataset, DataLoader')

###
### Display ###
###

_IMPORT_('import cv2')
_IMPORT_('from PIL import Image')
_IMPORT_('from torchvision.utils import make_grid')
_IMPORT_('import matplotlib.pyplot as plt')
_IMPORT_('import plotly')
_IMPORT_('import plotly.graph_objects as go')

# plotly.offline.init_notebook_mode(connected=False)

def show_video(video_path, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if video_path.startswith('http'):
        data_url = video_path
    else:
        mp4 = open(video_path, 'rb').read()
        data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
    return HTML('<video %s %s controls src="%s" type="video/mp4"/>' % (W, H, data_url))

def show_image(image_path, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if image_path.startswith('http'):
        data_url = image_path
    else:
        img = open(image_path, 'rb').read()
        data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

from k12libs.utils.nb_easy import RACEURL

In [31]:
API_INFERENCE = f'{RACEURL}/raceai/framework/inference'
API_POPMSG = f'{RACEURL}/raceai/private/popmsg'
MODEL_TASK = 'zmq.repnet_tf.inference'
MSGKEY = 'zmq.repnet_tf.test'
TEST_SAMPLES = {
    'test0': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人7+8_精简.mp4',
    'test1': 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet_test.mp4',
    'test2': 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet/repnet_static_test.mp4',
    'test3': 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet/repnet_mix_test.mp4',
    'test4': 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet/repnet_mix2_test.mp4',
    'test5': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人1.mp4',
    'test6': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人2.mp4',
    'test7': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人4.mp4',
    'test8': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人5.mp4',
    'test9': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人6.mp4',
    'test10': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人1脚部.mp4',
    'test11': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人1齿轮.mp4',
    'test12': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人5踩.mp4',
    'test13': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅双人1焊接.mp4',
    'test14': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/120s单人脚部操作+整理.mp4',
    'test15': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人7.mp4',
    'test16': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人7+8.mp4',
    'test17': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人8.mp4',
    'test18': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人7+8_精简.mp4',
    'test19': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人坐姿_长视频.mp4',
    'test20': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人坐姿遮挡.mp4',
    'test21': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人坐姿大遮挡.mp4',
    'test22': 'https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人坐姿双遮挡.mp4',
}
API_INFERENCE, API_POPMSG 

('http://116.85.5.40:9119/raceai/framework/inference',
 'http://116.85.5.40:9119/raceai/private/popmsg')

In [8]:
show_video(TEST_SAMPLES['test20'], width=500)

## 接口测试

### 输入

```json

{
    "task": "zmq.repnet_tf.inference",  // 任务类型, 不可修改
    "cfg": {
        "pigeon": {  // 信鸽, 由服务器自行添加字段, 异步数据输出时会携带这些数据返回, 其中"msgkey", "user_code"必须有
            "msgkey": "zmq.repnet_tf.test",  // 异步数据Redis消息队列key
            "user_code": "123", // 任务由谁触发
        },
        "video": "https://raceai.s3.didiyunapi.com/data/media/videos/repnet_test.mp4", // 视频地址
        "save_video": True,   // 是否保存处理后的视频 (开启后, 速度非常慢)
        "batch_size": 20,     // batch_size
        "threshold": 0.2,     // 模型预测分数阀值
        "in_threshold": 0.5,  // 模型预测帧是否在周期内的阀值
        "strides": [1, 2, 3, 4],     // 送入模型的帧跳跃步长, 1表示逐个帧, 2表示跳跃一帧..., 模型会自行选择步长最合适的
        "constant_speed": False,     // 视频周期运动速度是否常数
        "median_filter": True,       // 是否对周期检测做中值滤波
        "fully_periodic": False,     // 周期检测是否使用全周期(所有帧), 而不是由in_threshold指定的阀值过滤
        "rm_still": False,           // 是否剔除掉静态帧
        "area_rate_threshold": 0.003,// 当rm_still为True时, 当前后两针变化像素小于图片面积的area_rate_threshold, 认为是静态的
        "best_stride_video": False   // 是否保存模型使用的最好的stride的视频帧
    }
}
```

In [32]:
_TEMPLATE_ = '''{
    "task": "%s",
    "cfg": {
        "pigeon": {
            "msgkey": "%s",
            "user_code": "%s",
        },
        "video": "%s",
        "save_video": True,
        "batch_size": 20,
        "threshold": 0.2,
        "in_threshold": 0.5,
        "strides": [2,3,4,5],
        "constant_speed": False,
        "median_filter": True,
        "fully_periodic": False,
        "rm_still": False,
        "area_rate_threshold": 0.003,
        "best_stride_video": False
    }
}'''

cfgdatas = {user_code:  _TEMPLATE_ % (
    MODEL_TASK, MSGKEY,
    user_code, video_url
) for user_code, video_url in TEST_SAMPLES.items()}

print("Input:\n\n", cfgdatas['test7'])

Input:

 {
    "task": "zmq.repnet_tf.inference",
    "cfg": {
        "pigeon": {
            "msgkey": "zmq.repnet_tf.test",
            "user_code": "test7",
        },
        "video": "https://raceai.s3.didiyunapi.com/data/media/videos/test/小幅单人4.mp4",
        "save_video": True,
        "batch_size": 20,
        "threshold": 0.2,
        "in_threshold": 0.5,
        "strides": [2,3,4,5],
        "constant_speed": False,
        "median_filter": True,
        "fully_periodic": False,
        "rm_still": False,
        "area_rate_threshold": 0.003,
        "best_stride_video": False
    }
}


### 输出

```json
 {
    "pigeon": {
        "msgkey": "zmq.repnet_tf.test",
        "user_code": "123"
    },
    "task": "zmq.repnet_tf.inference",
    "errno": 0,
    "progress": 100.0,
    "target_mp4": "https://raceai.s3.didiyunapi.com/outputs/123/repnet_tf-target.mp4",
    "target_json": "https://raceai.s3.didiyunapi.com/outputs/123/repnet_tf-results.json"
}
```

In [33]:
def get_result():
    while True:
        result = json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
        if len(result) > 0 and 'progress' in result[-1]:
            progress = int(result[-1]['progress'])
            print_progress_bar(progress)
            if progress >= 100.0:
                # print('\nOutput:\n\n', json.dumps(result[-1], indent=4))
                return result[-1]
        time.sleep(1)

In [41]:
reqdata = eval(cfgdatas['test1'])
# reqdata['cfg']['pigeon']['user_code'] = 'test3_rmstill'
reqdata['cfg']['rm_still'] = True
reqdata['cfg']['best_stride_video'] = True
reqdata['cfg']['strides'] = [9, 11, 12, 13]
json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
json.loads(requests.post(url=API_INFERENCE, json=reqdata).text)
print(reqdata)
result = get_result()
result

{'task': 'zmq.repnet_tf.inference', 'cfg': {'pigeon': {'msgkey': 'zmq.repnet_tf.test', 'user_code': 'test1'}, 'video': 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet_test.mp4', 'save_video': True, 'batch_size': 20, 'threshold': 0.2, 'in_threshold': 0.5, 'strides': [9, 11, 12, 13], 'constant_speed': False, 'median_filter': True, 'fully_periodic': False, 'rm_still': True, 'area_rate_threshold': 0.003, 'best_stride_video': True}}
Progress: 100%: ▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋

{'pigeon': {'msgkey': 'zmq.repnet_tf.test', 'user_code': 'test1'},
 'task': 'zmq.repnet_tf.inference',
 'errno': 0,
 'progress': 100.0,
 'target_mp4': 'https://raceai.s3.didiyunapi.com/outputs/test1/repnet_tf-target.mp4',
 'stride_mp4': 'https://raceai.s3.didiyunapi.com/outputs/test1/repnet_tf-target-stride.mp4',
 'target_json': 'https://raceai.s3.didiyunapi.com/outputs/test1/repnet_tf-results.json'}

In [26]:
json_result = requests.get(result['target_json']).json()
print(json.dumps(json_result, indent=4))

{
    "period": 147.21100758248042,
    "score": 0.6466431617736816,
    "stride": 9,
    "fps": 21.0,
    "num_frames": 4106,
    "num_still_frames": 2246,
    "area_rate_threshold": 0.003,
    "infer_time": 2.7791523933410645,
    "frames_period": [
        {
            "image_id": "0.jpg",
            "at_time": 0.048,
            "is_still": true,
            "within_period": 0.0,
            "pframe_counts": 0.0,
            "cum_counts": 0.0
        },
        {
            "image_id": "1.jpg",
            "at_time": 0.095,
            "is_still": false,
            "within_period": 0.5067752003669739,
            "pframe_counts": 0.02777777798473835,
            "cum_counts": 0.02777777798473835
        },
        {
            "image_id": "2.jpg",
            "at_time": 0.143,
            "is_still": true,
            "within_period": 0.0,
            "pframe_counts": 0.0,
            "cum_counts": 0.02777777798473835
        },
        {
            "image_id": "3.jpg",
     

In [ ]:
# show_video(json_result['target_mp4'])

## JSON结果数据

```json
{
    "period": 58.022187843507346,      // 视频周期平均值(帧数)
    "score": 0.6532081365585327,       // 视频预测的平均分数
    "stride": 2,                       // 最佳步长
    "fps": 29.916666666666668,         // 视频帧率
    "num_frames": 634,                 // 视频帧总数
    "num_still_frames": 7,             // 静止帧总数
    "area_rate_threshold": 0.003,      // 静止帧判断阀值
    "infer_time": 3.5414674282073975,  // 模型推理用时
    "mkvideo_time": 54.93384122848511, // 视频合成耗费的时间
    "frames_period": [                 // 每一帧的周期信息
        {
            "image_id": "0.jpg",                    // 第 1 帧
            "at_time": 0.033,                       // 该帧对应的视频中的时间数(秒)
            "is_still": false,                      // 是否为静止帧
            "within_period": 0.7064347267150879,    // 属于周期内的帧的置信度
            "pframe_counts": 0.019999999552965164,  // 每一帧所代表周期数量 (小数) 
            "cum_counts": 0.019999999552965164      // 循环周期次数依次累加
        },
        { 
            "image_id": "1.jpg",                    // 第 2 帧
            "at_time": 0.067,
            "is_still": false,                      // 是否为静止帧
            "within_period": 0.7064347267150879,
            "pframe_counts": 0.019999999552965164,
            "cum_counts": 0.03999999910593033
        },
        ....
        {
            "image_id": "392.jpg",                  // 第 393 帧
            "at_time": 13.111,
            "is_still": false,
            "within_period": 0.7592176795005798,
            "pframe_counts": 0.02083333395421505,
            "cum_counts": 5.636037701740861
        },
        {
            "image_id": "393.jpg",
            "at_time": 13.145,
            "is_still": false,
            "within_period": 0.7592176795005798,
            "pframe_counts": 0.02083333395421505,
            "cum_counts": 5.656871035695076
        },
      ....
    ],
    "target_mp4": "https://raceai.s3.didiyunapi.com/outputs/test3/repnet_tf-target.mp4" // 合成视频的存放地址
    "stride_mp4": "https://raceai.s3.didiyunapi.com/outputs/test3/repnet_tf-target-stride.mp4" // 最好stride合成视频的存放地址
}
```

## References

小幅单人2.mp4

    https://raceai.s3.didiyunapi.com/outputs/test6/repnet_tf-results.json
    https://raceai.s3.didiyunapi.com/outputs/test6/alphapose-results.json
    
小幅单人4.mp4:

    https://raceai.s3.didiyunapi.com/outputs/test7/repnet_tf-results.json
    https://raceai.s3.didiyunapi.com/outputs/test7/alphapose-results.json
    
小幅单人5.mp4:

    https://raceai.s3.didiyunapi.com/outputs/test8/repnet_tf-results.json
    https://raceai.s3.didiyunapi.com/outputs/test8/alphapose-results.json

小幅单人6.mp4:

    https://raceai.s3.didiyunapi.com/outputs/test9/repnet_tf-results.json
    https://raceai.s3.didiyunapi.com/outputs/test9/alphapose-results.json

小幅单人1脚部.mp4

    https://raceai.s3.didiyunapi.com/outputs/test10/repnet_tf-results.json
    https://raceai.s3.didiyunapi.com/outputs/test10/alphapose-results.json

小幅单人1齿轮.mp4

    https://raceai.s3.didiyunapi.com/outputs/test11/repnet_tf-results.json
    https://raceai.s3.didiyunapi.com/outputs/test11/alphapose-results.json

小幅单人5踩.mp4

    https://raceai.s3.didiyunapi.com/outputs/test12/repnet_tf-results.json
    https://raceai.s3.didiyunapi.com/outputs/test12/alphapose-results.json

小幅双人1焊接.mp4

    https://raceai.s3.didiyunapi.com/outputs/test13/repnet_tf-results.json
    https://raceai.s3.didiyunapi.com/outputs/test13/alphapose-results.json

120s单人脚部操作+整理.mp4

    https://raceai.s3.didiyunapi.com/outputs/test14/repnet_tf-results.json
    https://raceai.s3.didiyunapi.com/outputs/test14/alphapose-results.json

小幅单人7

    https://raceai.s3.didiyunapi.com/outputs/test15/repnet_tf-results.json

小幅单人7+8

    https://raceai.s3.didiyunapi.com/outputs/test16/repnet_tf-results.json

小幅单人8

    https://raceai.s3.didiyunapi.com/outputs/test17/repnet_tf-results.json 
    
小幅单人7+8_精简.mp4

    https://raceai.s3.didiyunapi.com/outputs/test18/repnet_tf-results.json 
    
小幅单人坐姿_长视频.mp4

    https://raceai.s3.didiyunapi.com/outputs/test19/repnet_tf-results.json